X Variables = Aggregate the following values by search date/time 
    videoTitleSentimentScore_pos = vader pos label for video title
    captionSentimentScore_pos = vader pos label for caption string
    viewcounts_weighted = views count / number of day since published
    like_count_weighted = like count / number of day since published
    dislike_count_weighted = dislike count / number of day since published
    like_dislike_ratio = like / (like + dislike)
    like_dislike_ratio_by_viewcounts_weighted = like dislike ration * viewcount_weight
    
Y Variable (Classifier) *Need to investigate %change
    Hold = 0 (Precent change 0% - 3%)
    Buy = 1 (Precent change > 3%)
    Sell = 2 ((Precent change < -3%)
        Interval of 5 days
        
Y Variable (Numerical)
    Future percent change (1Day)
    Future percent change (3Day)
    Future percent change (5Day)
    Future percent change (7Day)
    Future percent change (14Day)

In [108]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

### Feature Built Data Set load

In [167]:
#load cleaned and feature built data set
nvidia_sent_df = pd.read_csv("../00_data/Cleaned_Sent_Data_16MAR_v3.gz", compression="gzip") #, index_col = "Date")

In [168]:
nvidia_sent_df = nvidia_sent_df.drop(["Date"], axis=1)

ValueError: invalid literal for int() with base 10: 'buy'

In [63]:
nvidia_sent_df.action.value_counts()

hold    56
buy     19
sell    17
Name: action, dtype: int64

In [169]:
test_slip_percentage = .30
split_loc = int(len(nvidia_sent_df) - (len(nvidia_sent_df)* test_slip_percentage))
X = nvidia_sent_df.iloc[0:,1:-4]
X_train = nvidia_sent_df.iloc[0:split_loc,1:-4]
y_train = nvidia_sent_df.iloc[0:split_loc,-2]
X_test = nvidia_sent_df.iloc[split_loc:,1:-4]
y_test = nvidia_sent_df.iloc[split_loc:,-2]

In [170]:
enc = LabelEncoder()
enc.fit(y_train.values.reshape(-1, 1))
y_train = enc.transform(y_train.values)
y_test = enc.transform(y_test.values)

In [171]:
X_train

,videoTitleSentimentScore_pos,videoTitleSentimentScore_neg,captionSentimentScore_pos,captionSentimentScore_neg,num_days_since_pub,viewcounts_weighted,like_count_weighted,like_dislike_ratio,weighted_like_dislike_ratio,like_dislike_ratio_by_viewcounts_weighted
0,0.081317,0.099751,0.102556,0.036249,17.925926,71814.282680,3271.401019,0.969752,0.969752,69921.826086
1,0.059800,0.065880,0.116100,0.034660,35.600000,71921.099664,3523.220911,0.969585,0.969585,70150.404005
2,0.066000,0.059480,0.110600,0.035320,21.680000,66084.953130,3030.285674,0.972772,0.972772,64444.365869
3,0.071018,0.065469,0.110771,0.031495,19.829091,61208.718514,3150.355807,0.971021,0.971021,59725.066775
4,0.066353,0.069018,0.110309,0.031280,19.432727,72276.063635,3948.601316,0.972598,0.972598,70807.388299
...,...,...,...,...,...,...,...,...,...,...
59,0.045040,0.086400,0.101920,0.032360,95.920000,41010.916980,2254.516275,0.963893,0.963893,39788.786370
60,0.051540,0.113520,0.098160,0.033090,88.995000,39017.225744,2379.833176,0.969128,0.969128,38220.963827
61,0.055440,0.101793,0.114942,0.037342,90.261818,33842.598550,1983.632068,0.968902,0.968902,33139.467171
62,0.045040,0.107520,0.112120,0.037120,94.020000,28538.995293,1585.897619,0.963377,0.963377,27880.729753


In [172]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

C:\Users\Chris\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.32142857142857145

In [173]:
from sklearn import metrics
predictions = clf.predict(X_test)
cm = metrics.confusion_matrix(y_test, predictions)
print(cm)

[[0 3 7]
 [0 9 5]
 [0 4 0]]


In [174]:
predictions

array([1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1,
       2, 1, 1, 1, 1, 1])

In [175]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
scaler.transform(X)

array([[ 0.96268826,  1.33592613, -0.41470994,  1.64714757, -1.76319426,
         2.67904668,  2.02208384,  0.62900657,  0.62900657,  2.66970696],
       [-0.16502906,  0.43954573,  0.92960103,  1.32824275, -1.2152318 ,
         2.68468264,  2.2577217 ,  0.60341335,  0.60341335,  2.68205613],
       [ 0.15990926,  0.27017429,  0.38371594,  1.46072855, -1.64680391,
         2.37675123,  1.79646265,  1.09200291,  1.09200291,  2.3737812 ],
       [ 0.42290919,  0.42867131,  0.40067898,  0.69282185, -1.70418902,
         2.11946745,  1.90881703,  0.82352273,  0.82352273,  2.11881592],
       [ 0.17839549,  0.52259547,  0.35484268,  0.64975484, -1.71647777,
         2.70341153,  2.65576701,  1.06536026,  1.06536026,  2.71755042],
       [ 1.69340849, -0.3286243 ,  0.70198349, -0.18396897, -1.5594767 ,
         0.5855395 ,  0.43629496,  0.77574398,  0.77574398,  0.58954884],
       [ 2.62562889, -0.86461432,  1.1705123 , -1.79119576, -1.4934763 ,
        -1.06497992, -1.00040925,  0.77914989

In [176]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(scaler.transform(X_train), y_train)
clf.score(scaler.transform(X_test), y_test)

0.35714285714285715

In [177]:
from sklearn import metrics
predictions = clf.predict(scaler.transform(X_test))
cm = metrics.confusion_matrix(y_test, predictions)
print(cm)

[[0 5 5]
 [0 7 7]
 [0 1 3]]


In [178]:
predictions 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 1])

### Neural Network

In [185]:
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(max_iter=2000).fit(X_train, y_train)
inner_score = clf.score(X_test, y_test)
predictions = clf.predict(X_test)
cm = metrics.confusion_matrix(y_test, predictions)

In [186]:
inner_score 

0.5

In [187]:
cm

array([[ 0, 10,  0],
       [ 0, 14,  0],
       [ 0,  4,  0]], dtype=int64)

In [188]:
clf = MLPClassifier(max_iter=2000).fit(scaler.transform(X_train), y_train)
score = clf.score(scaler.transform(X_test), y_test)
predictions = clf.predict(scaler.transform(X_test))
cm = metrics.confusion_matrix(y_test, predictions)
print(score)
print(cm)

0.25
[[0 5 5]
 [1 6 7]
 [0 3 1]]


In [43]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(X_train)

ValueError: could not convert string to float: '2020-10-02'

In [39]:
X_train

,Unnamed: 0,Date,videoTitleSentimentScore_pos,videoTitleSentimentScore_neg,captionSentimentScore_pos,captionSentimentScore_neg,num_days_since_pub,viewcounts_weighted,like_count_weighted,like_dislike_ratio,weighted_like_dislike_ratio,like_dislike_ratio_by_viewcounts_weighted
0,0,2020-10-02,0.081317,0.099751,0.102556,0.036249,17.925926,71814.282680,3271.401019,0.969752,0.969752,69921.826086
1,1,2020-10-03,0.059800,0.065880,0.116100,0.034660,35.600000,71921.099664,3523.220911,0.969585,0.969585,70150.404005
2,2,2020-10-04,0.066000,0.059480,0.110600,0.035320,21.680000,66084.953130,3030.285674,0.972772,0.972772,64444.365869
3,3,2020-10-05,0.071018,0.065469,0.110771,0.031495,19.829091,61208.718514,3150.355807,0.971021,0.971021,59725.066775
4,4,2020-10-06,0.066353,0.069018,0.110309,0.031280,19.432727,72276.063635,3948.601316,0.972598,0.972598,70807.388299
...,...,...,...,...,...,...,...,...,...,...,...,...
64,64,2021-01-04,0.054760,0.077840,0.098600,0.031680,106.280000,23169.534231,1225.458127,0.966498,0.966498,22612.282283
65,65,2021-01-07,0.071760,0.086280,0.102160,0.035440,105.840000,19157.774784,1014.453700,0.966674,0.966674,18698.612308
66,66,2021-01-08,0.071760,0.081240,0.104640,0.032560,112.000000,19830.827916,1033.534572,0.966234,0.966234,19360.201979
67,67,2021-01-15,0.051720,0.076080,0.092070,0.029870,104.790000,31017.333485,1473.270963,0.958937,0.958937,29877.241452


In [ ]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(X_train)